In [ ]:

import pandas as pd
from qrpm.app.data_operations.log_overview import get_log_overview
from qrpm.analysis.quantityState import (
    determine_quantity_state_qel,
    determine_quantity_state_cp,
)
from numpy import mean, absolute
from qrpm.analysis.dataImport import load_qel_from_file
from analysis import * 
import math

In [ ]:
qel = load_qel_from_file(file_path=r"C:\Uni\7.Semester\QRPM\disqover\src\backend\files\Example_Inventory_Management_execution_log.sqlite")
qop = qel.get_quantity_operations()
print(qop)

In [ ]:
    # Ensure the "Time" column is in datetime format
qop["Time"] = pd.to_datetime(qop["Time"])
print(qop)

In [ ]:
qop["year_month"] = qop["Time"].dt.to_period("M")
print(qop)

In [ ]:
qop1 = qop.loc[qop["Tube"] < 0]
print(qop1)

In [ ]:
qop2 = qop1[['Tube', "year_month","Collection"]]
print(qop2)

In [ ]:
qop3 = qop2.loc[qop["Collection"].isin(['Company Warehouse'])]
print(qop3)

In [ ]:
demand = []
for period, group in qop.groupby("year_month"):
    monthly_demand = group['Tube'].sum()
    demand.append((str(period), -monthly_demand))
print(demand)

In [ ]:
import pandas as pd
from qrpm.app.data_operations.log_overview import get_log_overview
from qrpm.analysis.quantityState import (
    determine_quantity_state_qel,
    determine_quantity_state_cp,
)
from numpy import mean, absolute
from qrpm.analysis.dataImport import load_qel_from_file
qel = load_qel_from_file(file_path=r"C:\Uni\7.Semester\QRPM\disqover\src\backend\files\Example_Inventory_Management_execution_log.sqlite")


In [51]:
event_Place_Replenishment_Order = qel.get_event_data_activity("Place Replenishment Order")

print(event_Place_Replenishment_Order)

                          ocel_time
ocel_id                            
ev-2     2019-10-12 12:43:44.891246
ev-37    2019-10-14 03:33:26.520378
ev-54    2019-10-15 14:33:52.938551
ev-93    2019-10-17 06:59:28.150017
ev-163   2019-10-19 18:02:56.542362
...                             ...
ev-4313  2020-04-12 02:48:02.349542
ev-4329  2020-04-12 18:26:13.659650
ev-4342  2020-04-13 10:24:04.792542
ev-4419  2020-04-15 02:39:19.837236
ev-4492  2020-04-17 11:51:49.744839

[94 rows x 1 columns]


In [52]:
event_Put_Delivery_In_Stock = qel.get_event_data_activity("Place delivered Items into Inventory")
print (event_Put_Delivery_In_Stock)

                       ocel_time
ocel_id                         
ev-65    2019-10-16 10:10:10.100
ev-126   2019-10-17 10:10:10.100
ev-177   2019-10-19 10:10:10.100
ev-278   2019-10-23 10:10:10.100
ev-289   2019-10-24 10:10:10.100
...                          ...
ev-4409  2020-04-14 10:10:10.100
ev-4436  2020-04-15 10:10:10.100
ev-4469  2020-04-16 10:10:10.100
ev-4512  2020-04-18 10:10:10.100
ev-4525  2020-04-19 10:10:10.100

[98 rows x 1 columns]


In [53]:
event_object = qel.e2o
print(event_object)

     ocel_event_id ocel_object_id ocel_qualifier
0             ev-1            o-6           None
1             ev-1           o-10           None
2             ev-2           o-11           None
3             ev-2           o-21           None
4             ev-3           o-13           None
...            ...            ...            ...
5562       ev-4109         o-3387           None
5563       ev-4196         o-3402           None
5564       ev-4280         o-3407           None
5565       ev-4289         o-3483           None
5566       ev-4409         o-3536           None

[5567 rows x 3 columns]


In [80]:

rows = []
for event in event_Place_Replenishment_Order.index:
    temp_objects = qel.get_event_objects(event)
    for o in temp_objects.values:
        rows.append({"ro_id": o, "placed_time": event_Place_Replenishment_Order.loc[event, "ocel_time"]})
PlacedRO = pd.DataFrame(rows)
print(PlacedRO)

      ro_id                 placed_time
0      o-11  2019-10-12 12:43:44.891246
1      o-21  2019-10-12 12:43:44.891246
2      o-46  2019-10-14 03:33:26.520378
3      o-11  2019-10-14 03:33:26.520378
4      o-13  2019-10-15 14:33:52.938551
..      ...                         ...
183  o-3407  2020-04-13 10:24:04.792542
184    o-11  2020-04-15 02:39:19.837236
185  o-3483  2020-04-15 02:39:19.837236
186    o-11  2020-04-17 11:51:49.744839
187  o-3536  2020-04-17 11:51:49.744839

[188 rows x 2 columns]


In [81]:
rows = []
for event in event_Put_Delivery_In_Stock.index:
    temp_objects = qel.get_event_objects(event)
    for o in temp_objects.values:
        rows.append({"ro_id": o, "delivered_time": event_Put_Delivery_In_Stock.loc[event, "ocel_time"]})
DeliveredRO = pd.DataFrame(rows)
print(DeliveredRO)


     ro_id           delivered_time
0     o-21  2019-10-16 10:10:10.100
1     o-46  2019-10-17 10:10:10.100
2     o-59  2019-10-19 10:10:10.100
3     o-89  2019-10-23 10:10:10.100
4    o-146  2019-10-24 10:10:10.100
..     ...                      ...
89  o-3387  2020-04-03 10:10:10.100
90  o-3402  2020-04-06 10:10:10.100
91  o-3407  2020-04-10 10:10:10.100
92  o-3483  2020-04-11 10:10:10.100
93  o-3536  2020-04-14 10:10:10.100

[94 rows x 2 columns]


In [82]:
merged_df = pd.merge(
    DeliveredRO,
    PlacedRO,
    on="ro_id",
    how="inner",
    suffixes=("_delivered", "_placed")
)

# Print the result
print(merged_df)

     ro_id           delivered_time                 placed_time
0     o-21  2019-10-16 10:10:10.100  2019-10-12 12:43:44.891246
1     o-46  2019-10-17 10:10:10.100  2019-10-14 03:33:26.520378
2     o-59  2019-10-19 10:10:10.100  2019-10-15 14:33:52.938551
3     o-89  2019-10-23 10:10:10.100  2019-10-17 06:59:28.150017
4    o-146  2019-10-24 10:10:10.100  2019-10-19 18:02:56.542362
..     ...                      ...                         ...
89  o-3387  2020-04-03 10:10:10.100  2020-04-12 02:48:02.349542
90  o-3402  2020-04-06 10:10:10.100  2020-04-12 18:26:13.659650
91  o-3407  2020-04-10 10:10:10.100  2020-04-13 10:24:04.792542
92  o-3483  2020-04-11 10:10:10.100  2020-04-15 02:39:19.837236
93  o-3536  2020-04-14 10:10:10.100  2020-04-17 11:51:49.744839

[94 rows x 3 columns]


In [ ]:

# Calculate lead_time
merged_df["lead_time"] = merged_df["delivered_time"] - merged_df["placed_time"]
# Display the result
merged_df.to_csv("lead_time.csv")
print(merged_df)

     ro_id          delivered_time                placed_time  \
0     o-21 2019-10-16 10:10:10.100 2019-10-12 12:43:44.891246   
1     o-46 2019-10-17 10:10:10.100 2019-10-14 03:33:26.520378   
2     o-59 2019-10-19 10:10:10.100 2019-10-15 14:33:52.938551   
3     o-89 2019-10-23 10:10:10.100 2019-10-17 06:59:28.150017   
4    o-146 2019-10-24 10:10:10.100 2019-10-19 18:02:56.542362   
..     ...                     ...                        ...   
89  o-3387 2020-04-03 10:10:10.100 2020-04-12 02:48:02.349542   
90  o-3402 2020-04-06 10:10:10.100 2020-04-12 18:26:13.659650   
91  o-3407 2020-04-10 10:10:10.100 2020-04-13 10:24:04.792542   
92  o-3483 2020-04-11 10:10:10.100 2020-04-15 02:39:19.837236   
93  o-3536 2020-04-14 10:10:10.100 2020-04-17 11:51:49.744839   

                  lead_time  
0    3 days 21:26:25.208754  
1    3 days 06:36:43.579622  
2    3 days 19:36:17.161449  
3    6 days 03:10:41.949983  
4    4 days 16:07:13.557638  
..                      ...  
89 -9 day